In [1]:
from melbviz.pedestrian import PedestrianDataset
from melbviz.config import MELBVIZ_CLEANED_DATA_PATH
from ipywidgets import interact, Dropdown

In [2]:
def interactive_sensor_traffic(dataset):
    """Make an interactive"""
    year_widget = Dropdown(options=dataset.years)
    month_widget = Dropdown(options=dataset.months)
    sensor_widget = Dropdown(options=data.sensors)

    def update_widgets(*args):
        """Update month and sensor values to be only those available for selected year"""
        filtered_data = dataset.filter(year=year_widget.value)
        month_widget.options = filtered_data.months
        sensor_widget.options = filtered_data.sensors

    # register update_widgets as callback to be run on year change
    year_widget.observe(update_widgets)

    @interact(year=year_widget, month=month_widget, sensor=sensor_widget)
    def plot(year, month, sensor):
        """Plot the sensor traffic for selected year, month, and sensor"""
        filtered_data = dataset.filter(year=year, month=month, sensor=sensor)
        if len(filtered_data.df) == 0:
            return f"No records for {year}, {month}, {sensor}"
        return filtered_data.plot("sensor_traffic")

In [3]:
data = PedestrianDataset.from_parquet(MELBVIZ_CLEANED_DATA_PATH)

In [4]:
interactive_sensor_traffic(data)

interactive(children=(Dropdown(description='year', options=(2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 20…